## Import

In [92]:
%load_ext autoreload
%autoreload 2

import numpy as np

from initial_pop import *
from code_classes import *
from Operators.selection import *
from Operators.population import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Population

In [93]:
players = load_players_from_csv("data/players(in).csv")
players

[Alex Carter (GK) Skill: 85 Salary: €90.0M,
 Jordan Smith (GK) Skill: 88 Salary: €100.0M,
 Ryan Mitchell (GK) Skill: 83 Salary: €85.0M,
 Chris Thompson (GK) Skill: 80 Salary: €80.0M,
 Blake Henderson (GK) Skill: 87 Salary: €95.0M,
 Daniel Foster (DEF) Skill: 90 Salary: €110.0M,
 Lucas Bennett (DEF) Skill: 85 Salary: €90.0M,
 Owen Parker (DEF) Skill: 88 Salary: €100.0M,
 Ethan Howard (DEF) Skill: 80 Salary: €70.0M,
 Mason Reed (DEF) Skill: 82 Salary: €75.0M,
 Logan Brooks (DEF) Skill: 86 Salary: €95.0M,
 Caleb Fisher (DEF) Skill: 84 Salary: €85.0M,
 Nathan Wright (MID) Skill: 92 Salary: €120.0M,
 Connor Hayes (MID) Skill: 89 Salary: €105.0M,
 Dylan Morgan (MID) Skill: 91 Salary: €115.0M,
 Hunter Cooper (MID) Skill: 83 Salary: €85.0M,
 Austin Torres (MID) Skill: 82 Salary: €80.0M,
 Gavin Richardson (MID) Skill: 87 Salary: €95.0M,
 Spencer Ward (MID) Skill: 84 Salary: €85.0M,
 Sebastian Perry (FWD) Skill: 95 Salary: €150.0M,
 Xavier Bryant (FWD) Skill: 90 Salary: €120.0M,
 Elijah Sanders 

In [94]:
population = generate_population(players, num_leagues=10)

for i, league in enumerate(population):
    print(f"\n--- League {i+1} ---")
    print(league)
    print(f"Standard Deviation of Avg Skills: {league.get_standard_deviation_of_average_skills():.2f}")


--- League 1 ---
Ryan Mitchell (GK) Skill: 83 Salary: €85.0M
Caleb Fisher (DEF) Skill: 84 Salary: €85.0M
Logan Brooks (DEF) Skill: 86 Salary: €95.0M
Austin Torres (MID) Skill: 82 Salary: €80.0M
Bentley Rivera (MID) Skill: 88 Salary: €100.0M
Adrian Collins (FWD) Skill: 85 Salary: €90.0M
Elijah Sanders (FWD) Skill: 93 Salary: €140.0M

Blake Henderson (GK) Skill: 87 Salary: €95.0M
Mason Reed (DEF) Skill: 82 Salary: €75.0M
Daniel Foster (DEF) Skill: 90 Salary: €110.0M
Spencer Ward (MID) Skill: 84 Salary: €85.0M
Dylan Morgan (MID) Skill: 91 Salary: €115.0M
Colton Gray (FWD) Skill: 91 Salary: €125.0M
Chase Murphy (FWD) Skill: 86 Salary: €95.0M

Alex Carter (GK) Skill: 85 Salary: €90.0M
Owen Parker (DEF) Skill: 88 Salary: €100.0M
Jaxon Griffin (DEF) Skill: 79 Salary: €65.0M
Hunter Cooper (MID) Skill: 83 Salary: €85.0M
Connor Hayes (MID) Skill: 89 Salary: €105.0M
Xavier Bryant (FWD) Skill: 90 Salary: €120.0M
Tyler Jenkins (FWD) Skill: 80 Salary: €70.0M

Jordan Smith (GK) Skill: 88 Salary: €10

## Test Selection Functions

In [95]:
import random
from collections import Counter

def test_selection(population, selector_fn, selector_name, trials=100):
    picks = []
    for _ in range(trials):
        selected = selector_fn(population)
        # identify by fitness and index
        idx = population.index(selected)
        fitness = calculate_fitness(selected)
        picks.append((idx, round(fitness, 4)))
    counter = Counter(picks)

    print(f"\n{selector_name} results over {trials} trials:")
    for (idx, fit), count in counter.most_common():
        print(f"  League #{idx} (fitness={fit}): picked {count} times")


if __name__ == "__main__":
    # 1) Create a pool of, say, 50 players with random skills & costs
    positions = ["GK"]*10 + ["DEF"]*15 + ["MID"]*15 + ["FWD"]*10
    all_players = [
        Player(
            name=f"P{i}",
            position=random.choice(positions),
            skill=random.uniform(50, 100),
            cost=random.uniform(50, 200),
        )
        for i in range(1, 51)
    ]

### Roulette

In [96]:
test_selection(population, roulette_selection, "Roulette‐Wheel Selection")


Roulette‐Wheel Selection results over 100 trials:
  League #8 (fitness=0.4371): picked 18 times
  League #3 (fitness=1.1794): picked 15 times
  League #5 (fitness=0.8495): picked 12 times
  League #1 (fitness=1.5581): picked 12 times
  League #0 (fitness=0.9536): picked 10 times
  League #9 (fitness=1.119): picked 8 times
  League #4 (fitness=1.3199): picked 7 times
  League #2 (fitness=1.8701): picked 6 times
  League #6 (fitness=1.4774): picked 6 times
  League #7 (fitness=1.4411): picked 6 times


### Tournament

In [97]:
test_selection(
    population,
    lambda population: tournament_selection(population, tournament_size=3),
    "Tournament Selection (k=3)"
    )


Tournament Selection (k=3) results over 100 trials:
  League #8 (fitness=0.4371): picked 32 times
  League #0 (fitness=0.9536): picked 25 times
  League #5 (fitness=0.8495): picked 21 times
  League #3 (fitness=1.1794): picked 9 times
  League #9 (fitness=1.119): picked 7 times
  League #4 (fitness=1.3199): picked 4 times
  League #7 (fitness=1.4411): picked 1 times
  League #6 (fitness=1.4774): picked 1 times


### Stochastic

In [98]:
from collections import Counter

def test_stochastic(population, selector_fn, selector_name, trials=100):
    counter = Counter()
    for _ in range(trials):
        selected_list = selector_fn(population)  # this is a list of num_parents Leagues
        for sel in selected_list:
            # find which original index this copy corresponds to
            idx = next(i for i, L in enumerate(population)
                       if calculate_fitness(L) == calculate_fitness(sel))
            counter[idx] += 1

    print(f"\n{selector_name} results over {trials} trials (total picks = {sum(counter.values())}):")
    for idx, count in counter.most_common():
        fit = round(calculate_fitness(population[idx]), 4)
        print(f"  League #{idx} (fitness={fit}): picked {count} times")


In [99]:
test_stochastic(
    population,
    lambda population: stochastic_selection(population, num_parents=5),
    "Stochastic Universal Sampling (N=5)",
    trials=100
)



Stochastic Universal Sampling (N=5) results over 100 trials (total picks = 500):
  League #8 (fitness=0.4371): picked 124 times
  League #5 (fitness=0.8495): picked 64 times
  League #0 (fitness=0.9536): picked 59 times
  League #3 (fitness=1.1794): picked 50 times
  League #9 (fitness=1.119): picked 43 times
  League #4 (fitness=1.3199): picked 38 times
  League #7 (fitness=1.4411): picked 34 times
  League #1 (fitness=1.5581): picked 34 times
  League #6 (fitness=1.4774): picked 34 times
  League #2 (fitness=1.8701): picked 20 times
